In [4]:
# 기본 세팅

from urllib.request import Request, urlopen
from urllib.parse import quote, urlencode, quote_plus, unquote
import json
from bs4 import BeautifulSoup as bs

import copy
import os
import datetime as dt
import time as t
import numpy as np
import pandas as pd
import random as rd

from tqdm.notebook import tqdm as tq

# 날짜 지정

In [5]:
'''1. 목표 날짜 지정  (검색할 날짜)'''
# 검색 시작 날짜    
first_date = 20200220
current_date = (int(str(first_date)[:4]) , int(str(first_date)[4:6]), int(str(first_date)[6:]))

# 검색 종료 날짜
end_date = 202002024

# 크롤링

In [13]:
########################################
##### 1. url 크롤링                #####
########################################


##############
data_set = {}          # 데이터가 담길 dictionary
############## 


# first_date 에서 날짜 넘겨받는 부분  (시작점)
current_date = (int(str(first_date)[:4]) , int(str(first_date)[4:6]), int(str(first_date)[6:]))
current_date_WHILE = int(dt.datetime(current_date[0], current_date[1], current_date[2]).strftime('%Y%m%d'))


'''1. 날짜 하루씩 추가하면서 크롤링 진행'''
# '날짜 지정'에서 설정한 검색종료 날짜까지 작업 반복
while True :
    if current_date_WHILE > end_date:
        break
    print(current_date_WHILE)
    
    '''
    2. current_date 를 기반으로 날짜 변수를 datetime 형태로 저장  -
    ->    이후의 날짜 계산을 용이하게 하기 위함
    '''
    s_date = dt.datetime(current_date[0], current_date[1], current_date[2]) 
    e_date = s_date + dt.timedelta(days=1)   

    '''
    3. datetime 날짜의 type 조작   -->    검색창에 입력할 수 있는 형태로 변환
    '''

    # 1.  20200201  형태로 저장
    str_s_date1 = s_date.strftime('%Y%m%d')
    str_e_date1 = e_date.strftime('%Y%m%d')

    # 2,  2020.02.01 형태로 저장
    str_s_date2 = s_date.strftime('%Y.%m.%d')
    str_e_date2 = e_date.strftime('%Y.%m.%d')


    '''
    4. url 및 검색 parameter 지정
    '''

    # base url
    base_url = 'https://search.naver.com/search.naver'

    # parameters
    query_params = '?' + urlencode(
        {
            quote_plus('where') : 'news',
            quote_plus('query') : "+코로나 창업 | 스타트업",
            quote_plus('sm') : 'tab_opt',
            quote_plus('') : '',
            quote_plus('sort')  : '2',
            quote_plus('photo') : '0',
            quote_plus('field') : '0',
            quote_plus('pd') : '3',
            quote_plus('ds') : str_s_date2,
            quote_plus('de') : str_e_date2,
            quote_plus('docid') : '',  
            quote_plus('related') : '0',
            quote_plus('mynews') : '1',       # 언론사 선택시 1 / default = 0 
            quote_plus('office_type') :  '1',  # default = 0 / 일간지 = 1 
            quote_plus('office_section_code') : '1',  # 언론사 선택하면 1, default = 0
            
            ########################
            ### 신문사 코드 입력 ###
            ########################
            
            quote_plus('news_office_checked') : '1032',  #경향 1032,  만일 없으면 공백으로 비워두어야 함
        }
    )


    # 첫번째 반복시행에서 article index는 1 고정
    first_article_index = 1


    ###############################  특정 날짜에서의 검색창 크롤링 ###########################
    #iteration_idx = 0
    while True:    

        try:
            #debug code
            #iteration_idx +=1
            #print('while 작업 횟수 : ', iteration_idx)

            '''
            5. 뉴스 검색창 접속
            '''
            # 기타 parameter
            start = '&start=' + str(first_article_index)  # 페이지 첫번째 기사의 index, 
                                    # 처음은 1로 시작하나 페이지 넘길 때마다 10씩 증가시켜야 함
                                    # 즉, 날짜별 iteration 코드에 < start += 10 >  를 넣어주어야 함
            query_others = '&nso=so%3Ar%2Cp%3Afrom' + str_s_date1 + 'to' + str_e_date1 + '&is_sug_officeid=0' + start

            # url
            url = base_url + query_params + query_others
            # 서버에 접속. 너무 급한 속도의 접속은 IP ban을 유발할 수 있음
            req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            page = urlopen(req)
            
        except:
            t.sleep(20)

            '''
            5. 뉴스 검색창 접속
            '''
            # 기타 parameter
            start = '&start=' + str(first_article_index)  # 페이지 첫번째 기사의 index, 
                                    # 처음은 1로 시작하나 페이지 넘길 때마다 10씩 증가시켜야 함
                                    # 즉, 날짜별 iteration 코드에 < start += 10 >  를 넣어주어야 함
            query_others = '&nso=so%3Ar%2Cp%3Afrom' + str_s_date1 + 'to' + str_e_date1 + '&is_sug_officeid=0' + start

            # url
            url = base_url + query_params + query_others
            # 서버에 접속. 너무 급한 속도의 접속은 IP ban을 유발할 수 있음
            req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            page = urlopen(req)

            
        t.sleep(0.3)     # IP ban 방지용
        

        
        '''
        6. page parsing
        '''
        
        # bs parsing
        page = bs(page, 'html.parser')

        # IS_EMPTY 는 검색결과가 있다면 [] 를 반환함. 
        # 만일 IS_EMPTY 가 []가 아니라면 검색결과가 없는 것이므로 break 해야 함.
        IS_EMPTY = page.find_all('div', 'api_noresult_wrap')
        if IS_EMPTY != []:
            break 

        # 필요 데이터 parsing (url, 날짜)
        url_resultset = page.find_all('div', 'news_info')   #  --> url
        news_date_resultset = page.find_all('div', 'info_group')  # --> 날짜,  둘다 parsing 후의 index는 같음



        '''
        7. 지면뉴스 오류 handling
        '''
        for idx in range(len(url_resultset)):

            # url
            target_url = url_resultset[idx].a['data-url']

            # 날짜
            if news_date_resultset[idx].i:
                IS_PAPERNEWS = True
            else:
                IS_PAPERNEWS = False

            if IS_PAPERNEWS ==False:
                target_date = news_date_resultset[idx].find('span', 'info').text

            else:
                try:
                    target_date = news_date_resultset[idx].find_all('span', 'info')[1].text
                except:
                    target_date = news_date_resultset[idx].find_all('span', 'info')[0].text


            # 데이터 저장, key = url  / value = date
            data_set[target_url] = target_date




        # 다음 검색 페이지로 넘어가기 위해 기사 index 조정
        first_article_index += 10


    # 반복문 break에 사용할 current_date 설정
    current_date = s_date
    current_date += dt.timedelta(days = 1)  # 하루 더하기
    current_date = current_date.strftime('%Y%m%d')  # 더한 날짜정보를 str으로 바꾸기

    # str으로 변환된 날짜정보를 연,월,일로 분리
    year = current_date[:4]
    month = current_date[4:6]
    day = current_date[6:]

    # 다음 iteration에 넘어갈 current_date 정보
    current_date = (int(year), int(month), int(day))


    # 날짜 while문 종료에 관한 정보
    current_date_WHILE = int(dt.datetime(current_date[0], current_date[1], current_date[2]).strftime('%Y%m%d'))
    
    
    
'''
데이터 내보내기 (url)
'''     
        
table = pd.DataFrame(data_set.keys(), data_set.values())
table = table.drop_duplicates()
table.to_csv('경향 url주소.csv', encoding = 'cp949')

t.sleep(2)


########################################
##### 2. 본문 크롤링               #####
########################################


# url 가져오기 (csv파일)
urls = list(pd.read_csv('경향 url주소.csv', encoding = 'cp949')['0'])



# 데이터가 들어갈 공간
data_set = {}


'''
data columns :
    date   / title  / body   / company  / url
    (날짜) / (제목) / (본문) / (언론사) / (url)

'''


#url 접속 후 크롤링


for idx in range(len(urls)):
    
    # 차단 막기 위해 잠시 휴식
    #t.sleep(0.3)
    
    # 초기화
    body = ''
    title = ''
    day = ''
    company = ''

    
   # 경향
    try:
        req = Request(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
        )
        t.sleep(0.3)
        page = urlopen(req)
        page = bs(page, 'html.parser')
    except:
        t.sleep(20)
        req = Request(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
        )
        t.sleep(0.3)
        page = urlopen(req)
        page = bs(page, 'html.parser')
                               
    ### 데이터 저장 ###

    body = page.find("meta", {"property":"og:description"}).get("content")
    title = page.find_all('title')[0].text
    day = page.find("meta", {"property":"dable:item_id"}).get("content")[:8]
    company = '경향'
    data_set[idx] = [day, title, body, company, urls[idx]]


        
'''
데이터 내보내기 (뉴스본문)
'''
        
dataframe = pd.DataFrame(data_set).transpose()
dataframe.columns = ['date', 'title', 'body', 'company', 'url']
# 본문 내 특수문자 제거
dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\xa0-\xff]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u1ecf]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u0001-\u0019]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u0021-\u9FBF]', repl = r'', regex=True)
dataframe['body']= dataframe['body'].str.replace(pat=r'[\u2026-\u2027]', repl = r'', regex=True)
dataframe['title']= dataframe['title'].str.replace(pat=r'[\u2026-\u2027]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(' b ', '')

dataframe = dataframe.reset_index()
dataframe.to_csv('경향{}to{}.csv'.format(first_date, end_date), encoding = "cp949")

20200220
20200221
20200222
20200223
20200224
20200225
20200226
20200227
20200228
20200229
20200301
20200302
20200303
20200304


KeyboardInterrupt: 

In [42]:
print('마지막으로 접속 시도하던 url : ', urls[idx])

마지막으로 접속 시도하던 url :  http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002242117025&code=920501


In [49]:
urls = list(pd.read_csv('경향 url주소.csv', encoding = 'cp949')['0'])
urls

['http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002141032001&code=970204',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002172218015&code=910203',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002202031025&code=990100',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002210600001&code=960100',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002232041015&code=100401',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002232055015&code=990100',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002242117025&code=920501',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002262031035&code=990100',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002271550018&code=920100',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002282035035&code=990201',
 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202003021039001&code=350101']

In [6]:
url = 'http://news.khan.co.kr/kh_news/khan_art_view.html?artid=202002242117025&code=920501'
req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
)
t.sleep(0.3)
page = urlopen(req)
page = bs(page, 'html.parser')
page


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>타다와 ‘택시면허 총량제’ 3라운드는?</title>
<link href="https://www.khan.co.kr/article/202002242117025" id="canonical" rel="canonical"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="161649467207997" property="fb:pages"/><meta content="경향비즈 산업·통상 최신기사를 포함합니다." name="description"/>
<meta content="경향비즈, 경향신문, 비즈앤라이프, biznlife, biz.khan, 경제 뉴스, 마켓, 비즈, 경제일반, 금융, 재테크, 산업, 통상, 무역, 기업, 일자리, 취업, market, finance, bank" name="keywords">
<meta content="경향비즈, 경향신문, 비즈앤라이프, biznlife, biz.khan, 경제 뉴스, 마켓, 비즈, 경제일반, 금융, 재테크, 산업, 통상, 무역, 기업, 일자리, 취업, market, finance, bank" name="news_keywords"/>
<meta content="zq2DdlnnxyflplT386MVi__pnshbhbtBXhwucHuDhtg" name="google-site-verification"/>
<meta content="http://biz.khan.co

In [7]:
# page.find("meta", {"name":"keywords"}).get("content")
# title = page.find("meta", {"name":"title"}).get("content")

page.find_all('title')[0].text

'타다와 ‘택시면허 총량제’ 3라운드는?'

In [9]:
page.find("meta", {"property":"og:description"}).get("content")
# property="og:description"

'26일 국회 법제사법위원회의 여객자동차운수사업법 개정안 논의를 앞둔 상황에서 ‘택시면허 총량제’ 유지...'

In [35]:
# <div class="art_cont">
# <div class="art_body" itemprop="articleBody">
# <p class="content_text">

body = ""
for line in page.find_all('p', class_='content_text'):
    body += line.get_text()
body


'\n국토부 기조 유지 땐 ‘기여금 완화’ 등 업계에 추가 조치 필요 목소리\n26일 국회 법제사법위원회의 여객자동차운수사업법 개정안 논의를 앞둔 상황에서 ‘택시면허 총량제’ 유지 여부에 다시 관심이 쏠리고 있다. 업계에선 정부가 ‘신사업을 하려면 택시면허를 사야 한다’는 기조를 포기할 수 없다면, 택시면허 대량확보를 신규 업체에 약속하는 등의 추가 조치를 내놔야 한다는 목소리가 나온다.면허 총량제를 둘러싼 1차 공방은 타다의 ‘판정패’였다. 지난해 5월 이재웅 쏘카 대표가 페이스북에 택시면허를 사들여 영업하는 것은 혁신의 해법이 아니라고 하자, 네이버 공동창업자인 김정호 베어베터 대표는 “날로 먹으려 들면 안된다”고 비판했다. 면허를 사서 운영하는 택시와 불공정하게 경쟁한다는 취지다. 지난해 12월 국토교통부가 타다 측에 여객자동차운수사업법 개정안의 찬성을 요구했을 때에도, 이 대표는 “택시를 꿈꾸지 않는 기업에 택시면허를 기반으로 하라고 하니까 합의할 수 없다”고 맞섰다.지난 19일 1심 법원 판결 이후 전세는 역전되는 분위기다. 법원이 택시면허 없이 운영하는 타다를 합법으로 판단해서다. 이후 국토부가 현안대로 개정안을 유지하려 하자, 이 대표는 “타다는 국토부의 블랙리스트에라도 올라가 있는 거냐”며 공세를 이어갔다.공은 국회 법사위에 있지만, 국토부는 ‘택시면허를 사서 영업해야 한다’는 기조를 유지한다는 입장이다. 수십만대의 렌터카가 타다처럼 택시시장에 유입될 경우, 25만대의 택시면허로 연간 8조원을 벌어들이는 택시시장이 붕괴한다고 보기 때문이다.업계에선 정부가 현재 기조를 포기할 수 없다면, 타다 등 신규 업체에 대량의 택시면허 확보와 대폭 완화된 택시면허 구입금(기여금) 등을 명확히 하는 추가 조치가 필요하다고 본다.업계의 한 관계자는 “타다와 달리 택시와 손잡고 있는 업체는 해당 법안의 통과를 내심 바라고 있다”며 “정부의 추가조치 없이는 현재의 대결구도가 해소되기 어려울 것”이라고 말했다.한편 타다는 택시기사에게 차량 구매비용 500만원을 지원하

In [47]:
page.find("meta", {"property":"dable:item_id"}).get("content")[:8]
# <meta content="202002242117025" property="dable:item_id"/>
# day = page.find("meta", {"property":"og:description"}).get("content")[:8]

'20200224'